# Interpolation comparison

This Jupyter Notebook is intended to reduce the amount of time spent manually comparing results from interpolation tools in ArcGIS Pro. The notebook was created in May 2020. Questions can be sent to Kalle Jahn at [kjahn@contractor.usgs.gov](mailto:kjahn@contractor.usgs.gov).

In [1]:
import os
import shutil
import arcpy
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from datetime import datetime

arcpy.env.overwriteOutput=True

# Input variables

In [ ]:
# geodatabase within the project folder that contains your point feature classes
projGeodb = 'test4.gdb'
arcpy.env.workspace = os.path.join(projGeodb)

# point feature class name
inPoints = 'Combined_L1_L2'

# percentage training points
trainSize = 90

# data to be interpolated (column in attribute table)
zField = 'L1_Surface_Meters'

# select transformations for simple, ordinary, and universal kriging
# TransformationNormalScore is for simple only...
# you must at the very least have 'NONE' ...
krigTransform = ['TransformationNormalScore','NONE','TransformationBoxCox','TransformationLog']

# EBK: neighborhood radius taken from Geostat Wizard
ebkRadius = 205

# EBK: maximum number of points in local models
# note that larger max leads to longer runtime, but necessary for large datasets (>10,000 points)
maxLocalPoints = 100

# set snap raster file, or write None if you have no snap raster
snapRaster = None
arcpy.env.snapRaster = snapRaster

# if snap raster exits, raster cell size is pulled from snap raster
# otherwise generated rasters will have cell size entered manually below
manualCellSize = 38.1 # # MUST BE SAME UNITS AS THOSE IN POINT FEATURE CLASS!
    
# boundary for topo to raster
# set to file name, otherwise set to None if you aren't using a boundary
boundary = None

# Functions

## Pre-transformation data shift

In [ ]:
def shift_up(in_features,z_field):
    data = arcpy.da.FeatureClassToNumPyArray(in_features, field_names=[z_field],
                                                     skip_nulls=True)
    minimum = np.min(data[zField])

    if minimum <= 0.0:
        factor = 0.1 if minimum == 0 else abs(minimum)+0.1
        arcpy.CalculateField_management(in_features,'shifted',
                                        '!{}! + {}'.format(z_field,factor),
                                        field_type='DOUBLE')
    
    else:
        factor = None
    return factor

## Updating XML

In [ ]:
def updateKrigXML(filename,transform,semivariogram,optimize='ByCrossvalidation'):
    '''
    Updates the XML file with the given semivariogram and adds optimization as if
    user had pressed the "Optimize" button in Geostat Wizard.
    
    INPUTS:
     filename (XML file):
         The XML file to be updated.
     semivariogram:
         The desired semivariogram model type. Must be available in Geostat Wizard...
    OUTPUTS:
     Updated XML file.
    '''
    tree = ET.ElementTree(file=filename)
    root = tree.getroot()
    
    # add the optimize attribute
    root.set('optimize', optimize)
    
    # get krig type
    km = root.find('enum[@name="KrigingMethodType"]')
    dataset = tree.find('./items/item')

    # edit transform type
    tran = dataset.find('./model')
    tran.set('name',transform)
    param = tran.find('./value')
    if tran.attrib['name'] == 'TransformationBoxCox' and param == None:
        p = ET.SubElement(tran, "value",attrib={'name':'Parameter'}).text = '1'  
    
    # if simple kriging and None or Log or BoxCox, set the mean to auto
#     if km.text == 'Simple' and (tran.attrib['name'] == 'None' or tran.attrib['name'] == 'TransformationBoxCox' or tran.attrib['name'] == 'TransformationLog'):
    if km.text == 'Simple' and tran.attrib['name'] != 'TransformationNormalScore':
        mean = dataset.find('./value[@name="Mean"]')
        if mean == None:
            ET.SubElement(dataset, "value",attrib={'name':'Mean','auto':'true'}).text = '2.0'
        else:
            mean.set('auto','true')
                          
    # add "auto" to the DensitySkew if that is what was chosen.
    t = tree.find('./items/item/model')
    m = t.find('./enum[@name="ApproximationMethodType"]')
    if m != None and m.text == 'DensitySkew':
        t.find('./enum[@name="BaseDistribution"]').set('auto','true')
        t.find('./value[@name="Kernels"]').set('auto','true')
    
    # set the variogram model
    sm = root.find('./model/model/enum[@name="ModelType"]')
    sm.text = semivariogram
    
    # need to add another parameter for some of the models
    vm = root.find('./model/model[@name="VariogramModel"]')
    param = root.find('./model/model/value[@name="Parameter"]')
    atts = {"auto": "true", "name": "Parameter"}
    if semivariogram == 'Stable':
        if param == None:
            new = ET.SubElement(vm, "value",attrib=atts).text = '2'  
        else:
            param.text = '2'
    elif semivariogram == 'BesselK' or semivariogram == 'BesselJ':
        if param == None:
            new = ET.SubElement(vm, "value",attrib=atts).text = '9.999'  
        else:
            param.text = '9.999'
    else:
        if param != None:
            vm.remove(param)
        
    
    with open(filename, 'wb') as fh:
        tree.write(fh)
    fh.close()

## Cross validation of Kriging and EBK

In [ ]:
def crossVal(outLayer): 
    '''
    INPUTS:
     outLayer (Geostatistical Layer):
         The geostatistical layer to be analyzed.
    OUTPUTS:
     statistics list :
         The cross-validation statistics stored in list form.
    '''
    cvResult = arcpy.CrossValidation_ga(outLayer)
    
    meanErr = np.float(cvResult.meanError)
    meanStd = np.float(cvResult.meanStandardized)
    rmsErr = np.float(cvResult.rootMeanSquare)
    rmsStd = np.float(cvResult.rootMeanSquareStandardized)
    avgStd = np.float(cvResult.averageStandard)
    in90 = np.float(cvResult.percentIn90Interval)
    in95 = np.float(cvResult.percentIn95Interval)
    avgCRPS = np.float(cvResult.averageCRPS)
    
    return [outLayer,meanErr,meanStd,rmsErr,rmsStd,avgStd,in90,in95,avgCRPS]        

## Subsetting, Training, and Testing

In [ ]:
def subset(in_features,trainSize=90):
    '''
    TO DO
    
    '''
    trainPoints = in_features[:-4]+'_train_pts'+str(trainSize)
    testPoints = in_features[:-4]+'_test_pts'+str((100-trainSize))
    subsizeUnits = "PERCENTAGE_OF_INPUT"

    arcpy.SubsetFeatures_ga(in_features, 
                            trainPoints, 
                            testPoints,
                            trainSize, 
                            subsizeUnits)
    return trainPoints, testPoints

In [ ]:
def TrainTestKrig(train_points,
                  test_points,
                  z_field,
                  outLayer,
                  xml,
                  cell_size,
                  trainSize=90
                 ):
    '''
    TO DO
    
    '''
    # create GA layers with 90% training points
    # method for Kriging
    outLayerTrain = outLayer+'_train'
    newDatasetTrain = arcpy.GeostatisticalDatasets(xml)
    newDatasetTrain.dataset1 = train_points
    newDatasetTrain.dataset1Field = z_field
    arcpy.GACreateGeostatisticalLayer_ga(xml,newDatasetTrain, outLayerTrain)
    
#     # save outLayer to GA layer file
#     out_layer_file = '.\\geostat_layers\\'+outLayerTrain+'.lyrx'
#     arcpy.SaveToLayerFile_management(outLayerTrain, out_layer_file, 'RELATIVE')

    # GA to raster to points using outLayerTrain and 90% train points
    trainRaster = outLayer+'_train_raster'
    try:
        arcpy.GALayerToGrid_ga(outLayerTrain,trainRaster,cell_size=cell_size)
        # extract raster values
        trainResult = outLayer+'_train_result'
        arcpy.sa.ExtractValuesToPoints(train_points,trainRaster,trainResult)
        # export values to numpy array
        field_names = [z_field,'RASTERVALU']
        arrTrain = arcpy.da.FeatureClassToNumPyArray(trainResult, field_names=field_names,
                                                         skip_nulls=False)
        meanTrain = np.mean(abs(arrTrain[z_field] - arrTrain['RASTERVALU']))
        medianTrain = np.median(abs(arrTrain[z_field] - arrTrain['RASTERVALU']))
        arcpy.Delete_management(trainResult)
    except arcpy.ExecuteError:
        meanTrain = 'ExecuteError'
        medianTrain = 'ExecuteError'
    
    # GA to points using trainRaster and 10% test points
    try:
        # extract raster values
        testResult = outLayer+'_test_result'
        x = 'extract values'
        arcpy.sa.ExtractValuesToPoints(test_points,trainRaster,testResult)
        # export values to numpy array
        field_names = [z_field,'RASTERVALU']
        x = 'to array'
        arrTest = arcpy.da.FeatureClassToNumPyArray(testResult, field_names=field_names,
                                                         skip_nulls=False)
        meanTest = np.mean(abs(arrTest[z_field] - arrTest['RASTERVALU']))
        medianTest = np.median(abs(arrTest[z_field] - arrTest['RASTERVALU']))
        arcpy.Delete_management(testResult)
    except arcpy.ExecuteError:
        meanTest = 'ExecuteError '+x
        medianTest = 'ExecuteError '+x
    
    arcpy.Delete_management(outLayerTrain)
    if trainRaster != None: arcpy.Delete_management(trainRaster)
        
    # return results in list
    return [outLayer,meanTrain,medianTrain,meanTest,medianTest]

In [ ]:
def TrainTestEBK(train_points,
                 test_points,
                 z_field,
                 outLayer,
                 outLayerTrain,
                 cell_size
                ):
    '''
    TO DO
    
    '''    
    # GA to raster and points using outLayerTrain and 90% train points
    trainRaster = outLayer+'_train_raster'    
    try:
        arcpy.GALayerToGrid_ga(outLayerTrain,trainRaster,cell_size=cell_size)
        # extract raster values
        trainResult = outLayer+'_train_result'
        arcpy.sa.ExtractValuesToPoints(train_points,trainRaster,trainResult)
        # export values to numpy array
        field_names = [z_field,'RASTERVALU']
        arrTrain = arcpy.da.FeatureClassToNumPyArray(trainResult, field_names=field_names,
                                                         skip_nulls=False)
        meanTrain = np.mean(abs(arrTrain[z_field] - arrTrain['RASTERVALU']))
        medianTrain = np.median(abs(arrTrain[z_field] - arrTrain['RASTERVALU']))
        arcpy.Delete_management(trainResult)
    except arcpy.ExecuteError:
        meanTrain = 'ExecuteError'
        medianTrain = 'ExecuteError'

    # GA to raster to points using trainRaster and 10% test points
    try:
        # extract raster values
        testResult = outLayer+'_test_result'
        x = 'extract values'
        arcpy.sa.ExtractValuesToPoints(test_points,trainRaster,testResult)
        # export values to numpy array
        field_names = [z_field,'RASTERVALU']
        x = 'to array'
        arrTest = arcpy.da.FeatureClassToNumPyArray(testResult, field_names=field_names,
                                                         skip_nulls=False)
        meanTest = np.mean(abs(arrTest[z_field] - arrTest['RASTERVALU']))
        medianTest = np.median(abs(arrTest[z_field] - arrTest['RASTERVALU']))
        arcpy.Delete_management(testResult)
    except arcpy.ExecuteError:
        meanTest = 'ExecuteError '+x
        medianTest = 'ExecuteError '+x
    if trainRaster != None: arcpy.Delete_management(trainRaster)
    # return results in list
    return [outLayer,meanTrain,medianTrain,meanTest,medianTest]

## Automated Kriging

In [ ]:
def autoKrig(in_features,
             train_points,
             test_points,
             z_field,
             cell_size,
             transform = ['TransformationNormalScore','NONE','TransformationBoxCox','TransformationLog'],
             krigTypes = ['Simple','Ordinary','Universal'],
             semivariograms = ['Circular','Spherical','Exponential',
                               'Gaussian','Stable','BesselK',
                               'BesselJ','RationalQuadratic'],
             ):
    '''
    TO ADD 
    
    '''
    # empty lists to fill with results
    k_results = []
    subset_results = []
    
    # models broken up below because only simple kriging has NormalScore transform
    simModels = list(itertools.product(['Simple'],transform,semivariograms))
    if 'TransformationNormalScore' in transform: 
        new = transform.copy()
        new.remove('TransformationNormalScore')
    else:
        new = transform.copy()
    ordUnvModels = list(itertools.product(['Ordinary','Universal'],new,semivariograms))
    
    models = simModels + ordUnvModels
    
    for i in models:
        if i[0] == 'Ordinary':
            xml = 'KrigingOrd.xml'
        elif i[0] == 'Simple':
            xml = 'KrigingSim.xml'
        elif i[0] == 'Universal':
            xml = 'KrigingUnv.xml'
            
        outLayer = 'Kriging_{}_{}_{}'.format(i[0],i[1],i[2])
        print(datetime.now().strftime('%H:%M:%S')+' working on '+outLayer)

        # update the XML file with appropriate semivariogram and transformation 
        updateKrigXML(xml,i[1],i[2])

        # create new GA dataset
        newDataset = arcpy.GeostatisticalDatasets(xml)
        newDataset.dataset1 = in_features
        newDataset.dataset1Field = z_field
        # new GA layer with optimized parameters
        arcpy.GACreateGeostatisticalLayer_ga(xml, newDataset, outLayer)

        # save outLayer to GA layer file
        out_layer_file = '.\\'+inPoints[:-4]+'_geostat_layers\\'+outLayer+'.lyrx'
        arcpy.SaveToLayerFile_management(outLayer, out_layer_file, 'RELATIVE')

        # calculate statistics and append to list
        try:
            r = crossVal(outLayer)
            k_results.append(r)
        except:
            k_results.append(['ExeErr'] * 9)
        # train and test with subsets            
        sr = TrainTestKrig(train_points,test_points,z_field,outLayer,xml,cell_size)
        subset_results.append(sr)

        arcpy.Delete_management(outLayer)
    
    print('creating dataframes')
    # create pandas dataframes from lists
    df1 = pd.DataFrame(k_results,columns=['model','ME','MSDE','RMSE','RMSSDE','avgStd','in90','in95','avgCRPS'])
    df2 = pd.DataFrame(subset_results,columns=['model','Mean Err - training pts','Median Err - training pts',
                                                       'Mean Err - testing pts','Median Err - testing pts'])
    return df1, df2

## Automated EBK

In [ ]:
# EBK initial test
def autoEBK(in_features,
            train_points,
            test_points,
            z_field,
            cell_size,
            ebk_radius,
            maxLocalPoints,
            numberSemivariograms=100
           ):
    '''
    TO ADD 
    
    '''
    
    field_names = [z_field]
    checkNeg = arcpy.da.FeatureClassToNumPyArray(in_features, field_names=field_names,
                                                     skip_nulls=True)
    minimum = np.min(checkNeg[zField])
    transformations = ['NONE','EMPIRICAL'] if minimum <=0 else ['NONE', 'EMPIRICAL', 'LOGEMPIRICAL']
#     transformations = ['NONE']
    
    # Search neighborhood variables
    # Standard Circular
    angle = 0
    maxNeighbors = 15
    minNeighbors = 5
    sectorType = 'FOUR_SECTORS_SHIFTED'
    searchNeighbourhood = arcpy.SearchNeighborhoodStandardCircular(ebk_radius, angle, maxNeighbors,
                                                                   minNeighbors, sectorType)
    # parameters
    outputType = 'PREDICTION'
    quantileValue = None
    thresholdType = None
    probabilityThreshold = None
    overlapFactor = 1
    
    ebk_results = []
    subset_results = []
    
    # these semivariograms are for no transformation
    s1 = ['POWER','LINEAR','THIN_PLATE_SPLINE']
    # these semivariograms are for empirical and logempirical transformations
    s2 = ['EXPONENTIAL','EXPONENTIAL_DETRENDED','WHITTLE','WHITTLE_DETRENDED','K_BESSEL','K_BESSEL_DETRENDED']
    
    # create model combinations list
    models = list(itertools.product(transformations[0:1],s1)) + \
             list(itertools.product(transformations[1:],s2))
    
    for i in models:
        outLayer = 'EBK_{}_{}'.format(i[0],i[1])
        print(datetime.now().strftime('%H:%M:%S')+' working on '+outLayer)
        
        arcpy.EmpiricalBayesianKriging_ga(in_features=in_features,
                                  z_field=z_field,
                                  out_ga_layer=outLayer,
                                  out_raster=None,
                                  cell_size=None,
                                  transformation_type=i[0],
                                  max_local_points=maxLocalPoints,
                                  overlap_factor=overlapFactor,
                                  number_semivariograms=numberSemivariograms,
                                  search_neighborhood=searchNeighbourhood, 
                                  output_type=outputType, 
                                  quantile_value=quantileValue, 
                                  threshold_type=thresholdType, 
                                  probability_threshold=probabilityThreshold,
                                  semivariogram_model_type=i[1]
                                 )
        
        # save GA layer file
        out_layer_file = '.\\'+inPoints[:-4]+'_geostat_layers\\'+outLayer+'.lyrx'
        arcpy.SaveToLayerFile_management(outLayer, out_layer_file, 'RELATIVE')

        # Cross-validation
        r = crossVal(outLayer)
        ebk_results.append(r)

        # train and test with subsets
        # create training layer
        outLayerTrain = outLayer+'_train'
        arcpy.EmpiricalBayesianKriging_ga(in_features=train_points,
                                          z_field=z_field,
                                          out_ga_layer=outLayerTrain,
                                          out_raster=None,
                                          cell_size=None,
                                          transformation_type=i[0],
                                          max_local_points=maxLocalPoints,
                                          overlap_factor=overlapFactor,
                                          number_semivariograms=numberSemivariograms,
                                          search_neighborhood=searchNeighbourhood, 
                                          output_type=outputType, 
                                          quantile_value=quantileValue, 
                                          threshold_type=thresholdType, 
                                          probability_threshold=probabilityThreshold,
                                          semivariogram_model_type=i[1]
                                         )

        sr = TrainTestEBK(train_points,test_points,z_field,outLayer,outLayerTrain,cell_size)
        subset_results.append(sr)
        
        arcpy.Delete_management(outLayer)
        arcpy.Delete_management(outLayerTrain)
                
    print('creating dataframes')
    # create pandas dataframes from lists
    df1 = pd.DataFrame(ebk_results,columns=['model','ME','MSDE','RMSE','RMSSDE','avgStd','in90','in95','avgCRPS'])
    df2 = pd.DataFrame(subset_results,columns=['model','Mean Err - training pts','Median Err - training pts',
                                                       'Mean Err - testing pts','Median Err - testing pts'])

    
    return df1, df2

## Topo to raster

In [ ]:
def ttr(trainPoints,
        testPoints,
        z_field,
        cell_size,
        boundary
       ):
        
    inPointElevations = '{} {} POINTELEVATION'.format(trainPoints,z_field)
    if boundary == None:
        outRaster = inPoints[:-4]+'topo_no_bound'
        arcpy.TopoToRaster_3d(inPointElevations, outRaster, cell_size=cell_size, data_type='SPOT')
    else:
        outRaster = inPoints[:-4]+'topo_bound'
        inBoundary = '{} # BOUNDARY'.format(boundary)
        inFeats = (inPointElevations+';'+inBoundary)
        arcpy.TopoToRaster_3d(inFeats, outRaster, cell_size=cell_size, data_type='SPOT') 

    # compare raster to 90% test points
    trainResult = inPoints[:-4]+'_topoTrain'
    arcpy.sa.ExtractValuesToPoints(trainPoints,outRaster,trainResult)
    
    field_names = [z_field,'RASTERVALU']
    arrTrain = arcpy.da.FeatureClassToNumPyArray(trainResult, field_names=field_names,
                                                     skip_nulls=True)
    meanTrain = np.mean(abs(arrTrain[z_field] - arrTrain['RASTERVALU']))
    medianTrain = np.median(abs(arrTrain[z_field] - arrTrain['RASTERVALU']))
    
    # compare raster to 10% test points
    testResult = inPoints[:-4]+'_topoTest'
    arcpy.sa.ExtractValuesToPoints(testPoints,outRaster,testResult)
    
    arrTest = arcpy.da.FeatureClassToNumPyArray(testResult, field_names=field_names,
                                                     skip_nulls=True)
    meanTest = np.mean(abs(arrTest[z_field] - arrTest['RASTERVALU']))
    medianTest = np.median(abs(arrTest[z_field] - arrTest['RASTERVALU']))
    
    arcpy.Delete_management(outRaster)
    arcpy.Delete_management(trainResult)
    arcpy.Delete_management(testResult)
    
    return [outRaster,meanTrain,medianTrain,meanTest,medianTest]

## Automated raster

In [ ]:
def autoRaster(trainPoints,
               testPoints,
               z_field,
               cell_size,
               boundary
              ):
        
    results = []
    
    # NATURAL NEIGHBORS
    print(datetime.now().strftime('%H:%M:%S')+' working on Natural Neighbor')
    outNat = arcpy.sa.NaturalNeighbor(trainPoints,z_field,cell_size)
    
    # compare raster to 90% test points
    trainResult = inPoints[:-4]+'_natTrain'
    arcpy.sa.ExtractValuesToPoints(trainPoints,outNat,trainResult)
    
    field_names = [z_field,'RASTERVALU']
    arrTrain = arcpy.da.FeatureClassToNumPyArray(trainResult, field_names=field_names,
                                                     skip_nulls=True)
    meanTrain = np.mean(abs(arrTrain[z_field] - arrTrain['RASTERVALU']))
    medianTrain = np.median(abs(arrTrain[z_field] - arrTrain['RASTERVALU']))
    
    # compare raster to 10% test points
    testResult = inPoints[:-4]+'_natTest'
    arcpy.sa.ExtractValuesToPoints(testPoints,outNat,testResult)
    
    arrTest = arcpy.da.FeatureClassToNumPyArray(testResult, field_names=field_names,
                                                     skip_nulls=True)
    meanTest = np.mean(abs(arrTest[z_field] - arrTest['RASTERVALU']))
    medianTest = np.median(abs(arrTest[z_field] - arrTest['RASTERVALU']))

    results.append(['Natural_Neighbor',meanTrain,medianTrain,meanTest,medianTest])
    
    arcpy.Delete_management(trainResult)
    arcpy.Delete_management(testResult)
    
    # TOPO TO RASTER
    print(datetime.now().strftime('%H:%M:%S')+' working on Topo to Raster without boundary')
    nobound = ttr(trainPoints,testPoints,z_field,cell_size,boundary=None)
    results.append(nobound)
    
    if boundary != None:
        print(datetime.now().strftime('%H:%M:%S')+' working on Topo to Raster with boundary')
        bound = ttr(trainPoints,testPoints,z_field,cell_size,boundary)
        results.append(bound)
        
    # prepare dataframe
    df = pd.DataFrame(results,columns=['model','Mean Err - training pts','Median Err - training pts',
                                          'Mean Err - testing pts','Median Err - testing pts'])
    
    return df

# Execution cells

## Data shift, train/test points, set cell size

In [ ]:
# normalize the data set if necessary and change zField if shift occurs
factor = shift_up(inPoints,zField)
print('normalize factor = {}'.format(factor))
if factor != None:
    zField = 'shifted'

In [ ]:
# create training and testing points
trainPoints, testPoints = subset(inPoints, trainSize)

In [ ]:
# select cell size based on snapraster
if snapRaster != None:
    cellSize = np.float(arcpy.GetRasterProperties_management(snapRaster, 'CELLSIZEY').getOutput(0))
else:
    cellSize = manualCellSize

## Interpolation 

In [ ]:
# Kriging methods (remove Bessel models if they create an error)
k1, k2 = autoKrig(inPoints,trainPoints,testPoints,zField,cellSize,krigTransform)

In [ ]:
# Empirical Bayesian Kriging
ebk1, ebk2 = autoEBK(inPoints,trainPoints,testPoints,zField,cellSize,ebkRadius,maxLocalPoints)

In [ ]:
# Topo to Raster and Natural Neighbors
raster = autoRaster(trainPoints,testPoints,zField,cellSize,boundary)

In [ ]:
# DATAFRAME CONCATINATION 
stats = pd.concat([k1,ebk1],sort=False)#.sort_values(by='RMSE')
sub_tests = pd.concat([k2,ebk2,raster],sort=False)#.sort_values(by='Mean Err - testing pts')
f = pd.DataFrame([factor], columns=['factor'],index=[''])

In [ ]:
# DATAFRAME EXPORT TO EXCEL
with pd.ExcelWriter(inPoints[:-4]+'_interpol.xlsx') as writer: # use this to create new file
    stats.to_excel(writer, sheet_name='Statistics')
    sub_tests.to_excel(writer, sheet_name='Subset_comparison')
    f.to_excel(writer, sheet_name='normalize_factor')
writer.save()
print('saved to Excel file')